## Import Libraries

In [1]:
import pandas as pd
import glob
import os

## Read Config File

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
geo_aus_file = config['DEFAULT']['Geo-Aus-File']
geo_world_file = config['DEFAULT']['Geo-World-File']
input_folder = config['DEFAULT']['Input-Folder']
output_folder = config['DEFAULT']['Output-Folder']

## Read Geoname

In [2]:
geo_aus = pd.read_csv(geo_aus_file, header=0, encoding="UTF-8",usecols=[0],low_memory=False)
geo_world = pd.read_csv(geo_world_file, header=0, encoding="UTF-8",usecols=[0],low_memory=False)
geo_aus_ulocation = geo_aus["user_location"].tolist()
geo_world_ulocation = geo_world["user_location"].tolist()

## Supporting Functions

In [ ]:
def get_new_user_location(user_location, geo_user_location):
    new_uer_location = []
    for uloc in user_location:
        if uloc not in geo_user_location:
            new_uer_location.append(uloc)
    return new_uer_location

In [ ]:
# create folder if not exist
def create_folder(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)

## Main Function

In [3]:
# create output folder if not exist
create_folder(output_folder)

for file in glob.glob(input_folder + "/*.csv"):
    # Read user location
    collection = pd.read_csv(file, header=0, encoding="UTF-8",engine="python")
    user_location = collection["user_location"].tolist()
    
    # get new user_location
    if "Australia" in file:
        new_uer_location = get_new_user_location(user_location, geo_aus_ulocation)
        country = "Australia"
    else:
        new_uer_location = get_new_user_location(user_location, geo_world_ulocation)
        country = "Other"
            
    # write into CSV
    df_output = pd.DataFrame(new_uer_location,columns=['user_location'])
    df_output.to_csv("{}/new_user_location_{}.csv".format(output_folder,country), sep=',',index = False, encoding='UTF-8')
    